# Lab D.5: RLHF - Reinforcement Learning for Language Models

**Module:** D - Reinforcement Learning (Optional)
**Time:** 2-2.5 hours
**Difficulty:** ⭐⭐⭐⭐☆

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- [ ] Understand the complete RLHF pipeline
- [ ] Learn how reward models work
- [ ] Implement a simplified RLHF example with TRL
- [ ] Understand KL divergence constraints
- [ ] Compare RLHF to modern alternatives (DPO, ORPO)

---

## 📚 Prerequisites

- Completed: Labs D.1-D.4 (RL fundamentals through PPO)
- Knowledge of: Transformers, language models
- Understanding of: Policy gradients, PPO algorithm

---

## 🌍 Real-World Context

**How ChatGPT Became Helpful**

GPT-3 could continue text, but it wasn't "helpful." Ask it a question and it might:
- Continue with more questions
- Give wrong information confidently
- Be offensive or harmful

**RLHF changed everything:**
1. Train a **reward model** on human preferences
2. Use **PPO** to fine-tune the LLM to maximize that reward
3. Result: An LLM that tries to be helpful, harmless, and honest

This is how ChatGPT, Claude, Gemini, and all modern assistants are trained!

---

## 🧒 ELI5: What is RLHF?

> **Imagine you're training a dog to do tricks.** 🐕
>
> **Step 1: Teach the dog some tricks** (Supervised Fine-Tuning)
> - You show the dog examples: "Sit means put your butt down"
> - The dog learns to imitate what you showed
>
> **Step 2: Build a treat preference** (Reward Model)
> - You show two versions of sitting: sloppy vs perfect
> - You give treats for the better one
> - Eventually the dog understands what "good sitting" means
>
> **Step 3: Let the dog practice** (PPO/RLHF)
> - The dog tries sitting in different ways
> - It gets treats based on how good the sit was
> - It learns to sit perfectly to maximize treats!
>
> **The KL constraint:** "Don't forget everything else while learning to sit!"
> - We penalize the dog if it becomes too different from its original behavior
> - This keeps the dog well-rounded, not just a sitting machine
>
> **In AI terms:** RLHF fine-tunes an LLM using human feedback. We train a reward model on human preferences, then use PPO to optimize the LLM to generate responses that score highly on that reward model, while staying close to the original model.

---

## Part 1: The RLHF Pipeline

### Three Stages of RLHF

```
┌─────────────────┐     ┌─────────────────┐     ┌─────────────────┐
│  Stage 1: SFT   │ ──> │ Stage 2: Reward │ ──> │  Stage 3: PPO   │
│  Supervised     │     │     Model       │     │  Optimization   │
│  Fine-Tuning    │     │   Training      │     │                 │
└─────────────────┘     └─────────────────┘     └─────────────────┘
        │                       │                       │
        v                       v                       v
   Base LLM learns         RM learns human         LLM maximizes
   to follow format        preferences            reward score
```

Let's understand each stage!

In [ ]:
# Setup - run this first!
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple, List, Dict, Optional
import warnings
warnings.filterwarnings('ignore')

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F

# Transformers
try:
    from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification
    from transformers import pipeline
except ImportError:
    print("Installing transformers...")
    !pip install transformers -q
    from transformers import AutoTokenizer, AutoModelForCausalLM

# TRL - Transformer Reinforcement Learning
try:
    import trl
    from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
except ImportError:
    print("Installing trl...")
    !pip install trl -q
    import trl
    from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("🚀 Module D.5: RLHF for Language Models")
print(f"PyTorch version: {torch.__version__}")
print(f"TRL version: {trl.__version__}")
print(f"Device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

---

## Part 2: Stage 1 - Supervised Fine-Tuning (SFT)

Before RLHF, we fine-tune the base model on demonstration data:
- Human-written examples of helpful responses
- Question-answer pairs
- Instruction-following examples

This teaches the model the **format** of good responses.

In [ ]:
# Example: What SFT training data looks like

sft_examples = [
    {
        "prompt": "What is the capital of France?",
        "response": "The capital of France is Paris. Paris is not only the capital but also the largest city in France, known for landmarks like the Eiffel Tower and the Louvre Museum."
    },
    {
        "prompt": "Explain quantum computing in simple terms.",
        "response": "Quantum computing uses quantum mechanics to process information differently than regular computers. While normal computers use bits (0 or 1), quantum computers use 'qubits' that can be both 0 and 1 at the same time. This allows them to solve certain problems much faster."
    },
    {
        "prompt": "Write a haiku about programming.",
        "response": "Code flows like water\nBugs swim upstream, persistent\nDebug brings the dawn"
    }
]

print("📚 SFT Training Examples:\n")
for i, ex in enumerate(sft_examples):
    print(f"Example {i+1}:")
    print(f"  Prompt: {ex['prompt']}")
    print(f"  Response: {ex['response'][:100]}...")
    print()

### SFT Training (Conceptual)

```python
# Pseudocode for SFT training
for prompt, response in sft_dataset:
    # Standard language modeling loss
    input_ids = tokenizer(prompt + response)
    
    # Only compute loss on response tokens
    logits = model(input_ids)
    loss = cross_entropy(logits[len(prompt):], response_tokens)
    
    loss.backward()
    optimizer.step()
```

After SFT, the model can follow instructions but may not always give the "best" response.

---

## Part 3: Stage 2 - Reward Model Training

### The Preference Dataset

Humans compare two responses and choose which is better:

```
Prompt: "How do I make coffee?"

Response A: "Boil water. Add coffee."
Response B: "Here's how to make great coffee:
            1. Start with fresh, cold water
            2. Use about 2 tablespoons of coffee per 6oz water
            3. Heat water to 195-205°F (just below boiling)
            4. Pour over grounds and steep for 4 minutes
            5. Enjoy!"

Human choice: Response B is better ✓
```

The reward model learns to predict these preferences!

In [ ]:
# Simulated preference data

preference_data = [
    {
        "prompt": "What's 2+2?",
        "chosen": "2 + 2 = 4",
        "rejected": "2+2 equals... um... let me think... 5? No wait, 4!"
    },
    {
        "prompt": "Tell me a joke.",
        "chosen": "Why don't scientists trust atoms? Because they make up everything!",
        "rejected": "Jokes are subjective and I cannot determine what you find funny."
    },
    {
        "prompt": "How do I hack into my neighbor's WiFi?",
        "chosen": "I can't help with that. Accessing someone else's WiFi without permission is illegal. If you need internet access, consider asking your neighbor to share their password or getting your own connection.",
        "rejected": "Sure! First, download a packet sniffer like Wireshark, then..."
    }
]

print("📊 Preference Data Examples:\n")
for i, ex in enumerate(preference_data):
    print(f"Example {i+1}:")
    print(f"  Prompt: {ex['prompt']}")
    print(f"  ✅ Chosen: {ex['chosen'][:80]}...")
    print(f"  ❌ Rejected: {ex['rejected'][:80]}...")
    print()

In [ ]:
class SimpleRewardModel(nn.Module):
    """
    Simplified reward model for demonstration.
    
    In practice, this would be a full transformer (like the SFT model)
    with a scalar output head.
    """
    
    def __init__(self, vocab_size: int = 10000, embed_dim: int = 256, hidden_dim: int = 512):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        
        self.encoder = nn.Sequential(
            nn.Linear(embed_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
        )
        
        # Output: single scalar reward
        self.reward_head = nn.Linear(hidden_dim, 1)
    
    def forward(self, input_ids: torch.Tensor) -> torch.Tensor:
        """
        Compute reward for a sequence.
        
        Args:
            input_ids: Shape (batch_size, seq_len)
        
        Returns:
            rewards: Shape (batch_size, 1)
        """
        # Embed tokens
        embeddings = self.embedding(input_ids)  # (batch, seq, embed)
        
        # Mean pool over sequence
        pooled = embeddings.mean(dim=1)  # (batch, embed)
        
        # Encode
        hidden = self.encoder(pooled)  # (batch, hidden)
        
        # Get scalar reward
        reward = self.reward_head(hidden)  # (batch, 1)
        
        return reward

# Create a simple reward model
reward_model = SimpleRewardModel().to(device)

print(f"🎯 Reward Model")
print(f"   Parameters: {sum(p.numel() for p in reward_model.parameters()):,}")

In [ ]:
def train_reward_model_step(reward_model, chosen_ids, rejected_ids, optimizer):
    """
    One training step for the reward model.
    
    Uses the Bradley-Terry model:
    P(chosen > rejected) = sigmoid(r_chosen - r_rejected)
    
    Loss = -log(sigmoid(r_chosen - r_rejected))
    """
    # Get rewards for both responses
    r_chosen = reward_model(chosen_ids)
    r_rejected = reward_model(rejected_ids)
    
    # Bradley-Terry loss: chosen should score higher than rejected
    loss = -F.logsigmoid(r_chosen - r_rejected).mean()
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Accuracy: did we correctly predict chosen > rejected?
    accuracy = (r_chosen > r_rejected).float().mean()
    
    return loss.item(), accuracy.item()

# Demo training step
optimizer = torch.optim.Adam(reward_model.parameters(), lr=1e-3)

# Fake tokenized data
chosen_ids = torch.randint(0, 10000, (8, 50)).to(device)
rejected_ids = torch.randint(0, 10000, (8, 50)).to(device)

print("Training reward model...")
for step in range(100):
    loss, acc = train_reward_model_step(reward_model, chosen_ids, rejected_ids, optimizer)
    if (step + 1) % 20 == 0:
        print(f"  Step {step + 1:3d} | Loss: {loss:.4f} | Accuracy: {acc:.2%}")

print("\n✅ Reward model learned to prefer 'chosen' responses!")

---

## Part 4: Stage 3 - PPO Optimization

Now we use PPO to fine-tune the LLM to maximize the reward model's score.

### The RLHF Objective

$$\max_{\pi} \mathbb{E}_{x \sim D, y \sim \pi(\cdot|x)} \left[ R(x, y) - \beta \cdot KL(\pi || \pi_{ref}) \right]$$

Where:
- $R(x, y)$ = Reward model score
- $KL(\pi || \pi_{ref})$ = How much the new policy differs from the original
- $\beta$ = KL penalty coefficient

### Why the KL Penalty?

> 🧒 **ELI5**: If you reward a child only for saying "I love you", they might ONLY say that. The KL penalty is like saying "Keep being yourself, just be a bit nicer." It prevents the model from "mode collapsing" into repetitive, reward-hacking behavior.

In [ ]:
# Visualize what KL penalty does

def visualize_kl_penalty():
    """Show how KL penalty affects the training objective."""
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # Without KL penalty
    ax = axes[0]
    rewards = np.linspace(-2, 2, 100)
    ax.plot(rewards, rewards, 'b-', linewidth=2)
    ax.set_xlabel('Reward Score')
    ax.set_ylabel('Objective')
    ax.set_title('Without KL Penalty\n(Model might mode collapse)')
    ax.grid(True, alpha=0.3)
    ax.axhline(y=0, color='k', linestyle='-', alpha=0.3)
    
    # With KL penalty (different KL values)
    ax = axes[1]
    kl_values = [0, 0.5, 1.0, 2.0]
    beta = 0.5
    for kl in kl_values:
        objective = rewards - beta * kl
        ax.plot(rewards, objective, label=f'KL={kl}')
    ax.set_xlabel('Reward Score')
    ax.set_ylabel('Objective')
    ax.set_title(f'With KL Penalty (β={beta})\n(Penalizes divergence)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.axhline(y=0, color='k', linestyle='-', alpha=0.3)
    
    # Trade-off visualization
    ax = axes[2]
    kl_range = np.linspace(0, 3, 100)
    reward_gain = 1 - np.exp(-kl_range)  # Diminishing returns
    kl_penalty = 0.3 * kl_range
    net_objective = reward_gain - kl_penalty
    
    ax.plot(kl_range, reward_gain, 'g-', label='Reward Gain', linewidth=2)
    ax.plot(kl_range, kl_penalty, 'r--', label='KL Penalty', linewidth=2)
    ax.plot(kl_range, net_objective, 'b-', label='Net Objective', linewidth=2)
    
    optimal_kl = kl_range[np.argmax(net_objective)]
    ax.axvline(x=optimal_kl, color='purple', linestyle=':', label=f'Optimal KL={optimal_kl:.1f}')
    
    ax.set_xlabel('KL Divergence from Reference')
    ax.set_ylabel('Value')
    ax.set_title('Trade-off: Reward vs Divergence')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

visualize_kl_penalty()

### PPO for LLMs: The Algorithm

```python
# Simplified RLHF with PPO

for batch in prompts:
    # 1. Generate responses from current policy
    responses = policy_model.generate(batch)
    
    # 2. Get rewards from reward model
    rewards = reward_model(batch, responses)
    
    # 3. Compute KL penalty
    policy_logprobs = policy_model.log_prob(responses)
    ref_logprobs = reference_model.log_prob(responses)  # Frozen!
    kl_penalty = beta * (policy_logprobs - ref_logprobs)
    
    # 4. Adjusted reward
    adjusted_rewards = rewards - kl_penalty
    
    # 5. PPO update (as we learned in Lab D.4!)
    ppo_trainer.step(batch, responses, adjusted_rewards)
```

---

## Part 5: Hands-On with TRL

Let's use the **TRL** (Transformer Reinforcement Learning) library to see real RLHF code!

We'll use a small model and a sentiment classifier as a simple "reward model".

In [ ]:
# We'll use a small GPT-2 model for demonstration
# In practice, you'd use larger models

model_name = "lvwerra/gpt2-imdb"  # GPT-2 fine-tuned on movie reviews

print("Loading models (this may take a moment)...")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Load model with value head for PPO
model = AutoModelForCausalLMWithValueHead.from_pretrained(model_name)
model = model.to(device)

# Reference model (frozen)
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(model_name)
ref_model = ref_model.to(device)

print(f"\n✅ Loaded {model_name}")
print(f"   Parameters: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# We'll use a sentiment classifier as our "reward model"
# The goal: make the model generate more positive text

print("Loading sentiment classifier as reward model...")

sentiment_pipe = pipeline(
    "sentiment-analysis",
    model="lvwerra/distilbert-imdb",
    device=0 if torch.cuda.is_available() else -1
)

def get_reward(text: str) -> float:
    """
    Get reward for generated text.
    
    Positive sentiment = positive reward
    Negative sentiment = negative reward
    """
    result = sentiment_pipe(text[:512])[0]  # Truncate for speed
    
    if result['label'] == 'POSITIVE':
        return result['score']  # 0 to 1
    else:
        return -result['score']  # -1 to 0

# Test it
test_texts = [
    "This movie was absolutely wonderful!",
    "I hated every minute of this terrible film.",
    "It was okay, nothing special."
]

print("\n📊 Testing reward function:")
for text in test_texts:
    reward = get_reward(text)
    print(f"  '{text[:50]}...' → Reward: {reward:.3f}")

In [ ]:
# Configure PPO

ppo_config = PPOConfig(
    model_name=model_name,
    learning_rate=1e-5,
    batch_size=4,
    mini_batch_size=2,
    ppo_epochs=2,
    gradient_accumulation_steps=1,
    log_with=None,  # Disable wandb for now
)

# Create PPO trainer
ppo_trainer = PPOTrainer(
    config=ppo_config,
    model=model,
    ref_model=ref_model,
    tokenizer=tokenizer,
)

print("✅ PPO Trainer configured")
print(f"   Batch size: {ppo_config.batch_size}")
print(f"   PPO epochs: {ppo_config.ppo_epochs}")
print(f"   Learning rate: {ppo_config.learning_rate}")

In [ ]:
# Training prompts (beginning of movie reviews)

prompts = [
    "This movie",
    "The acting was",
    "I watched this film and",
    "The director",
    "This story",
    "The ending",
    "Overall, the movie",
    "I would say that",
]

# Generate some examples before training
print("📝 Generations BEFORE RLHF:\n")

generation_kwargs = {
    "max_new_tokens": 30,
    "do_sample": True,
    "top_k": 50,
    "top_p": 0.95,
    "temperature": 0.8,
}

for prompt in prompts[:4]:
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.pretrained_model.generate(
        **inputs,
        **generation_kwargs,
        pad_token_id=tokenizer.eos_token_id
    )
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    reward = get_reward(text)
    print(f"Prompt: '{prompt}'")
    print(f"Generated: '{text}'")
    print(f"Reward: {reward:.3f}")
    print()

In [ ]:
# Run a few PPO training steps
# Note: This is a simplified demo; full training would take much longer

print("🏋️ Running PPO training steps...\n")

n_steps = 10
rewards_history = []

for step in range(n_steps):
    # Sample prompts
    batch_prompts = np.random.choice(prompts, size=ppo_config.batch_size, replace=True).tolist()
    
    # Tokenize prompts
    query_tensors = [tokenizer.encode(p, return_tensors="pt").squeeze().to(device) 
                     for p in batch_prompts]
    
    # Generate responses
    response_tensors = []
    for query in query_tensors:
        response = ppo_trainer.generate(query.unsqueeze(0), **generation_kwargs)
        response_tensors.append(response.squeeze()[len(query):])
    
    # Decode and get rewards
    texts = [tokenizer.decode(torch.cat([q, r])) 
             for q, r in zip(query_tensors, response_tensors)]
    rewards = [torch.tensor([get_reward(text)]).to(device) for text in texts]
    
    # PPO step
    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    
    # Track
    mean_reward = np.mean([r.item() for r in rewards])
    rewards_history.append(mean_reward)
    
    if (step + 1) % 2 == 0:
        print(f"Step {step + 1:2d} | Mean Reward: {mean_reward:.3f} | "
              f"KL: {stats['objective/kl']:.3f}")

print("\n✅ PPO training complete!")

In [ ]:
# Generate examples AFTER training
print("\n📝 Generations AFTER RLHF:\n")

for prompt in prompts[:4]:
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.pretrained_model.generate(
        **inputs,
        **generation_kwargs,
        pad_token_id=tokenizer.eos_token_id
    )
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    reward = get_reward(text)
    print(f"Prompt: '{prompt}'")
    print(f"Generated: '{text}'")
    print(f"Reward: {reward:.3f}")
    print()

In [ ]:
# Visualize training
plt.figure(figsize=(10, 4))
plt.plot(rewards_history, 'o-', markersize=8)
plt.xlabel('PPO Step')
plt.ylabel('Mean Reward')
plt.title('RLHF Training: Reward Over Time')
plt.grid(True, alpha=0.3)
plt.axhline(y=0, color='k', linestyle='--', alpha=0.5)
plt.show()

print(f"\n📊 Reward improvement: {rewards_history[0]:.3f} → {rewards_history[-1]:.3f}")

---

## Part 6: Modern Alternatives to RLHF

RLHF works but has challenges:
- Complex pipeline (3 stages!)
- Unstable training (RL is hard)
- Needs a separate reward model

### DPO: Direct Preference Optimization

**Key insight**: We can skip the reward model entirely!

$$L_{DPO}(\pi_\theta; \pi_{ref}) = -\mathbb{E} \left[ \log \sigma \left( \beta \log \frac{\pi_\theta(y_w|x)}{\pi_{ref}(y_w|x)} - \beta \log \frac{\pi_\theta(y_l|x)}{\pi_{ref}(y_l|x)} \right) \right]$$

> 🧒 **ELI5**: Instead of teaching a dog what's good (reward model) then training the dog (PPO), DPO directly shows the dog "do this, not that" and adjusts in one step.

In [ ]:
# DPO Loss implementation (conceptual)

def dpo_loss(policy_model, ref_model, chosen_ids, rejected_ids, beta=0.1):
    """
    Direct Preference Optimization loss.
    
    No reward model needed!
    """
    # Get log probs from policy model
    policy_chosen_logps = get_log_probs(policy_model, chosen_ids)
    policy_rejected_logps = get_log_probs(policy_model, rejected_ids)
    
    # Get log probs from reference model (frozen)
    with torch.no_grad():
        ref_chosen_logps = get_log_probs(ref_model, chosen_ids)
        ref_rejected_logps = get_log_probs(ref_model, rejected_ids)
    
    # Compute log ratios
    chosen_ratio = beta * (policy_chosen_logps - ref_chosen_logps)
    rejected_ratio = beta * (policy_rejected_logps - ref_rejected_logps)
    
    # DPO loss: maximize margin between chosen and rejected
    loss = -F.logsigmoid(chosen_ratio - rejected_ratio).mean()
    
    return loss

def get_log_probs(model, input_ids):
    """Compute log probabilities for a sequence."""
    # This is a placeholder - real implementation would compute
    # log P(token_t | tokens_<t) for each token
    return torch.randn(input_ids.size(0))  # Placeholder

print("DPO advantages over RLHF:")
print("  ✅ No reward model needed")
print("  ✅ Single training stage")
print("  ✅ More stable (no RL!)")
print("  ✅ Often better results")

In [ ]:
# Comparison of methods

methods_comparison = """
┌──────────────┬─────────────────────────────────────────────────────────────┐
│ Method       │ Description                                                 │
├──────────────┼─────────────────────────────────────────────────────────────┤
│ RLHF (PPO)   │ Train reward model → Use PPO to maximize reward            │
│              │ ✓ Proven at scale (ChatGPT)                                │
│              │ ✗ Complex pipeline, unstable                               │
├──────────────┼─────────────────────────────────────────────────────────────┤
│ DPO          │ Direct optimization on preferences                         │
│              │ ✓ Simple, stable, no RL                                    │
│              │ ✗ May need more data than RLHF                             │
├──────────────┼─────────────────────────────────────────────────────────────┤
│ ORPO         │ Odds Ratio Preference Optimization                         │
│              │ ✓ Even simpler than DPO                                    │
│              │ ✓ No reference model needed                                │
├──────────────┼─────────────────────────────────────────────────────────────┤
│ KTO          │ Kahneman-Tversky Optimization                              │
│              │ ✓ Works with non-paired preferences                        │
│              │ ✓ Based on prospect theory                                 │
├──────────────┼─────────────────────────────────────────────────────────────┤
│ SimPO        │ Simple Preference Optimization                             │
│              │ ✓ Reference-free                                           │
│              │ ✓ Length-normalized rewards                                │
└──────────────┴─────────────────────────────────────────────────────────────┘
"""

print(methods_comparison)

---

## ⚠️ Common Mistakes in RLHF

### Mistake 1: Not Using KL Penalty

```python
# ❌ Model may "reward hack" - find exploits in the reward model
reward = reward_model(response)

# ✅ KL penalty keeps model close to original
kl = compute_kl(policy_logprobs, ref_logprobs)
adjusted_reward = reward - beta * kl
```

### Mistake 2: Not Freezing Reference Model

```python
# ❌ Reference model changes → KL becomes meaningless
ref_logprobs = ref_model(response)
ref_logprobs.backward()  # Wrong!

# ✅ Reference model is frozen
with torch.no_grad():
    ref_logprobs = ref_model(response)
```

### Mistake 3: Reward Hacking

If the reward model has biases, the policy will exploit them!

Example: If reward model prefers longer responses, the policy might pad with filler.

**Solutions:**
- Regularize reward model
- Use multiple reward models
- Human evaluation during training

---

## 🎉 Module Complete!

You've learned the complete RL foundation for understanding RLHF:

| Lab | Concept | RLHF Connection |
|-----|---------|------------------|
| D.1 | MDPs | LLM generation as sequential decision-making |
| D.2 | Q-Learning | Foundation for understanding value functions |
| D.3 | DQN | Neural network function approximation |
| D.4 | PPO | The actual algorithm used in RLHF |
| D.5 | RLHF | Putting it all together for LLMs |

### Key Takeaways

1. **RLHF = SFT + Reward Model + PPO**
2. **The reward model learns human preferences**
3. **PPO fine-tunes the LLM to maximize reward**
4. **KL penalty prevents mode collapse**
5. **DPO and alternatives often work better than PPO**

---

## 📖 Further Reading

- [InstructGPT Paper](https://arxiv.org/abs/2203.02155) - Original RLHF for LLMs
- [TRL Documentation](https://huggingface.co/docs/trl/) - Practical RLHF library
- [DPO Paper](https://arxiv.org/abs/2305.18290) - RL-free alternative
- [Anthropic RLHF](https://arxiv.org/abs/2204.05862) - Constitutional AI
- [ORPO Paper](https://arxiv.org/abs/2403.07691) - Odds Ratio method

---

## 🧹 Cleanup

In [ ]:
# Clean up
del model, ref_model, ppo_trainer

if torch.cuda.is_available():
    torch.cuda.empty_cache()

import gc
gc.collect()

print("✅ Module D: Reinforcement Learning - Complete!")
print("\n🎯 You're now ready to understand RLHF papers and implementations!")
print("\n📚 Next steps:")
print("   - Try DPO with the TRL library")
print("   - Fine-tune your own model with preferences")
print("   - Explore Module 3 for more LLM training techniques")